In [18]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('practice').getOrCreate()

In [19]:
### THINGS TO BE ADDED 

# -id    
# -department    
# -first    
# -last    
# -dob     
# -Joining_date 
# -age 
# -tenure(in days)
# -no_of_policies 
# -no_of_active_policy
# -no of_pending_policy
# -no_of_cancelled_policy
# -avg_income

In [20]:
df1=spark.read.csv('employee.csv',header=True)
df2=spark.read.csv('sales_data.csv',header=True)


In [21]:
from pyspark.sql.functions import *


# adding age
df1 = df1.withColumn("dob_formatted", to_date("dob", "dd/MM/yyyy"))
df1 = df1.withColumn("age", (datediff(current_date(), "dob_formatted") / 365.25).cast("int"))
df1 = df1.drop('dob_formatted')

df1.show()


+---+----------+--------+------+----------+------------+---+
| id|department|   first|  last|       dob|Joining_date|age|
+---+----------+--------+------+----------+------------+---+
|  1|     Sales|  Deepti|Sharma|01/01/1998|    20/02/22| 25|
|  2|     Sales|  Suresh|Mishra|28/05/1992|    31/05/17| 30|
|  3|     Sales|    Maya| patil|01/02/1998|    20/07/22| 25|
|  4|     Sales|Prashant|Jadhav|27/08/1989|    01/12/14| 33|
|  5|     Sales|  Deepak| Kumar|01/02/1998|    04/04/21| 25|
|  6|     Sales|  Pratik| Kumar|28/05/1993|    05/08/15| 29|
|  7|     Sales|   Mayur| Sinha|01/03/1998|    09/07/22| 24|
|  8|     Sales|    Amit|Sharma|26/08/1985|    10/09/09| 37|
|  9|     Sales|Prajakta| patil|12/03/1981|    18/05/06| 41|
| 10|     Sales|   Smita| Raina|28/05/1994|    12/08/16| 28|
+---+----------+--------+------+----------+------------+---+



In [22]:
# adding tenure (in days)
df1 = df1.withColumn("joiningdate_formatted", to_date("Joining_date", "dd/MM/yy"))
df1 = df1.withColumn("tenure (in days)", (datediff(current_date(), "joiningdate_formatted")).cast("int"))
df1 = df1.drop('joiningdate_formatted')

df1.show()


+---+----------+--------+------+----------+------------+---+----------------+
| id|department|   first|  last|       dob|Joining_date|age|tenure (in days)|
+---+----------+--------+------+----------+------------+---+----------------+
|  1|     Sales|  Deepti|Sharma|01/01/1998|    20/02/22| 25|             362|
|  2|     Sales|  Suresh|Mishra|28/05/1992|    31/05/17| 30|            2088|
|  3|     Sales|    Maya| patil|01/02/1998|    20/07/22| 25|             212|
|  4|     Sales|Prashant|Jadhav|27/08/1989|    01/12/14| 33|            3000|
|  5|     Sales|  Deepak| Kumar|01/02/1998|    04/04/21| 25|             684|
|  6|     Sales|  Pratik| Kumar|28/05/1993|    05/08/15| 29|            2753|
|  7|     Sales|   Mayur| Sinha|01/03/1998|    09/07/22| 24|             223|
|  8|     Sales|    Amit|Sharma|26/08/1985|    10/09/09| 37|            4908|
|  9|     Sales|Prajakta| patil|12/03/1981|    18/05/06| 41|            6119|
| 10|     Sales|   Smita| Raina|28/05/1994|    12/08/16| 28|    

In [23]:
idc=df2.groupBy('id ').agg(count('id ').alias('no. of policies'))
idc.show()

+---+---------------+
|id |no. of policies|
+---+---------------+
|  7|              1|
|  3|              9|
|  8|              4|
|  5|              3|
|  6|              4|
|  9|              6|
|  1|              8|
| 10|              5|
|  4|              5|
|  2|              6|
+---+---------------+



In [24]:
df1=df1.join(idc,df1.id ==  idc["id "] ,"inner")
df1=df1.drop("id ")
df1.show()


+---+----------+--------+------+----------+------------+---+----------------+---------------+
| id|department|   first|  last|       dob|Joining_date|age|tenure (in days)|no. of policies|
+---+----------+--------+------+----------+------------+---+----------------+---------------+
|  7|     Sales|   Mayur| Sinha|01/03/1998|    09/07/22| 24|             223|              1|
|  3|     Sales|    Maya| patil|01/02/1998|    20/07/22| 25|             212|              9|
|  8|     Sales|    Amit|Sharma|26/08/1985|    10/09/09| 37|            4908|              4|
|  5|     Sales|  Deepak| Kumar|01/02/1998|    04/04/21| 25|             684|              3|
|  6|     Sales|  Pratik| Kumar|28/05/1993|    05/08/15| 29|            2753|              4|
|  9|     Sales|Prajakta| patil|12/03/1981|    18/05/06| 41|            6119|              6|
|  1|     Sales|  Deepti|Sharma|01/01/1998|    20/02/22| 25|             362|              8|
| 10|     Sales|   Smita| Raina|28/05/1994|    12/08/16| 28|

In [25]:
ap=df2.groupBy('id ','status').agg(count(
  'id ').alias("no. of active policy")).filter(
  col('status') == 'In Force')
  
ap.show()

+---+--------+--------------------+
|id |  status|no. of active policy|
+---+--------+--------------------+
|  1|In Force|                   3|
|  5|In Force|                   1|
|  7|In Force|                   1|
|  8|In Force|                   1|
|  6|In Force|                   2|
| 10|In Force|                   2|
|  2|In Force|                   2|
|  4|In Force|                   1|
|  3|In Force|                   3|
|  9|In Force|                   1|
+---+--------+--------------------+



In [26]:
df1=df1.join(ap,df1.id ==  ap["id "] ,"left")
df1=df1.drop("id ")
df1=df1.drop("status")
df1.show()


+---+----------+--------+------+----------+------------+---+----------------+---------------+--------------------+
| id|department|   first|  last|       dob|Joining_date|age|tenure (in days)|no. of policies|no. of active policy|
+---+----------+--------+------+----------+------------+---+----------------+---------------+--------------------+
|  7|     Sales|   Mayur| Sinha|01/03/1998|    09/07/22| 24|             223|              1|                   1|
|  3|     Sales|    Maya| patil|01/02/1998|    20/07/22| 25|             212|              9|                   3|
|  8|     Sales|    Amit|Sharma|26/08/1985|    10/09/09| 37|            4908|              4|                   1|
|  5|     Sales|  Deepak| Kumar|01/02/1998|    04/04/21| 25|             684|              3|                   1|
|  6|     Sales|  Pratik| Kumar|28/05/1993|    05/08/15| 29|            2753|              4|                   2|
|  9|     Sales|Prajakta| patil|12/03/1981|    18/05/06| 41|            6119|   

In [27]:
pp=df2.groupBy('id ','status').agg(count(
  'id ').alias("no. of pending policy")).filter(
  col('status') == 'Pending')
  
pp.show()

+---+-------+---------------------+
|id | status|no. of pending policy|
+---+-------+---------------------+
|  4|Pending|                    2|
| 10|Pending|                    1|
|  6|Pending|                    1|
|  8|Pending|                    2|
|  3|Pending|                    4|
|  5|Pending|                    1|
|  2|Pending|                    4|
|  1|Pending|                    4|
|  9|Pending|                    4|
+---+-------+---------------------+



In [28]:
df1=df1.join(pp,df1.id ==  pp["id "] ,"left")
df1=df1.drop("id ")
df1=df1.drop("status")
df1.show()


+---+----------+--------+------+----------+------------+---+----------------+---------------+--------------------+---------------------+
| id|department|   first|  last|       dob|Joining_date|age|tenure (in days)|no. of policies|no. of active policy|no. of pending policy|
+---+----------+--------+------+----------+------------+---+----------------+---------------+--------------------+---------------------+
|  7|     Sales|   Mayur| Sinha|01/03/1998|    09/07/22| 24|             223|              1|                   1|                 null|
|  3|     Sales|    Maya| patil|01/02/1998|    20/07/22| 25|             212|              9|                   3|                    4|
|  8|     Sales|    Amit|Sharma|26/08/1985|    10/09/09| 37|            4908|              4|                   1|                    2|
|  5|     Sales|  Deepak| Kumar|01/02/1998|    04/04/21| 25|             684|              3|                   1|                    1|
|  6|     Sales|  Pratik| Kumar|28/05/199

In [29]:
cp=df2.groupBy('id ','status').agg(count(
  'id ').alias("no. of cancelled policy")).filter(
  col('status') == 'Cancelled')
  
cp.show()

+---+---------+-----------------------+
|id |   status|no. of cancelled policy|
+---+---------+-----------------------+
|  1|Cancelled|                      1|
|  8|Cancelled|                      1|
|  6|Cancelled|                      1|
|  5|Cancelled|                      1|
|  4|Cancelled|                      2|
| 10|Cancelled|                      2|
|  3|Cancelled|                      2|
|  9|Cancelled|                      1|
+---+---------+-----------------------+



In [30]:
df1=df1.join(cp,df1.id ==  cp["id "] ,"left")
df1=df1.drop("id ")
df1=df1.drop("status")
df1.show()


+---+----------+--------+------+----------+------------+---+----------------+---------------+--------------------+---------------------+-----------------------+
| id|department|   first|  last|       dob|Joining_date|age|tenure (in days)|no. of policies|no. of active policy|no. of pending policy|no. of cancelled policy|
+---+----------+--------+------+----------+------------+---+----------------+---------------+--------------------+---------------------+-----------------------+
|  7|     Sales|   Mayur| Sinha|01/03/1998|    09/07/22| 24|             223|              1|                   1|                 null|                   null|
|  3|     Sales|    Maya| patil|01/02/1998|    20/07/22| 25|             212|              9|                   3|                    4|                      2|
|  8|     Sales|    Amit|Sharma|26/08/1985|    10/09/09| 37|            4908|              4|                   1|                    2|                      1|
|  5|     Sales|  Deepak| Kumar|01

In [31]:
ai=df2.groupBy('id ').agg(avg('income').alias("average income"))
ai.show()

+---+-----------------+
|id |   average income|
+---+-----------------+
|  7|         454159.0|
|  3|663854.5555555555|
|  8|        388765.25|
|  5|         636665.0|
|  6|         726671.5|
|  9|         648181.5|
|  1|       561890.375|
| 10|         857388.6|
|  4|         641406.2|
|  2|433256.6666666667|
+---+-----------------+



In [32]:
df1=df1.join(ai,df1.id ==  ai["id "] ,"inner")
df1=df1.drop("id ")
df1.show()


+---+----------+--------+------+----------+------------+---+----------------+---------------+--------------------+---------------------+-----------------------+-----------------+
| id|department|   first|  last|       dob|Joining_date|age|tenure (in days)|no. of policies|no. of active policy|no. of pending policy|no. of cancelled policy|   average income|
+---+----------+--------+------+----------+------------+---+----------------+---------------+--------------------+---------------------+-----------------------+-----------------+
|  7|     Sales|   Mayur| Sinha|01/03/1998|    09/07/22| 24|             223|              1|                   1|                 null|                   null|         454159.0|
|  3|     Sales|    Maya| patil|01/02/1998|    20/07/22| 25|             212|              9|                   3|                    4|                      2|663854.5555555555|
|  8|     Sales|    Amit|Sharma|26/08/1985|    10/09/09| 37|            4908|              4|            

In [33]:
df1=df1.withColumn('id',df1.id.cast('int'))
df1=df1.orderBy("id")
df1.show()


+---+----------+--------+------+----------+------------+---+----------------+---------------+--------------------+---------------------+-----------------------+-----------------+
| id|department|   first|  last|       dob|Joining_date|age|tenure (in days)|no. of policies|no. of active policy|no. of pending policy|no. of cancelled policy|   average income|
+---+----------+--------+------+----------+------------+---+----------------+---------------+--------------------+---------------------+-----------------------+-----------------+
|  1|     Sales|  Deepti|Sharma|01/01/1998|    20/02/22| 25|             362|              8|                   3|                    4|                      1|       561890.375|
|  2|     Sales|  Suresh|Mishra|28/05/1992|    31/05/17| 30|            2088|              6|                   2|                    4|                   null|433256.6666666667|
|  3|     Sales|    Maya| patil|01/02/1998|    20/07/22| 25|             212|              9|            